In [2]:
import sys
sys.path.append("../")
from sgan.data.loader import data_loader


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

In [4]:
class Config:
    def __init__(self, obs_len=12, pred_len=8, delim="tab", skip=1, metric="meter", batch_size=16, loader_num_workers=1):
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.delim = delim
        self.skip = skip
        self.metric = metric
        self.batch_size = batch_size
        self.loader_num_workers = loader_num_workers

In [5]:
train_path = "/mnt/c/Users/ktwzj/code/data/nba/nba-cross.s025/train_sample"
train_path = "/scratch/gw2145/data/nba/nba-cross.s025/train_sample"
args = Config()
train_dset, train_loader = data_loader(args, train_path)

100%|██████████| 44/44 [01:10<00:00,  1.61s/it]


In [6]:
for batch in train_loader:
    obs_traj = batch[0]
    print(obs_traj.shape)
    break

torch.Size([12, 176, 2])


In [ ]:
field_width = 28.7
field_height = 15.2
scene = obs_traj[:, :11, :]
sampling_scale_factor = 50
sampling_resolution = (int(sampling_scale_factor * field_width), int(sampling_scale_factor * field_height) )
image_grid = np.zeros((11, sampling_resolution[1], sampling_resolution[0]))
image = np.zeros((sampling_resolution[1], sampling_resolution[0]), np.uint8)
def build_map_function(scale_factor):
    def f (x, y):
        return x * scale_factor, y * scale_factor
    return f
sampling_map_func = build_map_function(sampling_scale_factor)
# image = cv2.fromarray(image)
image
center_coordinates = (129, 30)
 
# Radius of circle
radius = 3
  
# Blue color in BGR
color = 128
  
# Line thickness of 2 px
thickness = -1
  
# Using cv2.circle() method
# Draw a circle with blue line borders of thickness of 2 px
agent = scene[:, 0, :]
for i in range(agent.shape[0]):
    x = agent[i,0].item()
    y = agent[i, 1].item()
#     print(x.item())
    x,y = sampling_map_func(x,y)
    center_coordinates = (int(x), int(y))
    image = cv2.circle(image, center_coordinates, radius, color, thickness)
blur = cv2.GaussianBlur(image,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)

blur = cv2.GaussianBlur(blur,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)
blur = cv2.GaussianBlur(blur,(5,5),0)
kernel = np.ones((3,3),np.uint8)

dilation = cv2.dilate(blur,kernel,iterations = 5)

plt.imshow(blur, cmap="gray")

plt.subplot(131),plt.imshow(image,cmap="gray"),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(blur, cmap="gray"),plt.title('Blurred')
plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(dilation, cmap="gray"),plt.title('Dilation')
plt.xticks([]), plt.yticks([])
plt.show()

# plt.show()
dilation

In [7]:

class TrajectoryDrawer:
    field_width = 28.7
    field_height = 15.2
    def __init__(self, sampling_scale_factor=50, target_size=[572, 572],drawing_mode="blur"):
        self.sampling_scale_factor = sampling_scale_factor
        self.drawing_mode=drawing_mode
        self.target_size = target_size
        self.agent_num = 11
        self.sampling_resolution = (int(sampling_scale_factor * TrajectoryDrawer.field_width), int(sampling_scale_factor * TrajectoryDrawer.field_height) )

    def transform_sampling_coords (self, x, y):
        return x * self.sampling_scale_factor, y * self.sampling_scale_factor
    
    def generate_trajectory_image(self, agent, image):
        for i in range(agent.size(0)):
            x = agent[i,0].item()
            y = agent[i, 1].item()
            x,y = self.transform_sampling_coords(x,y)
            center_coordinates = (int(x), int(y))
            radius = 3
            color = 128
            thickness = -1
            image = cv2.circle(image, center_coordinates, radius, color, thickness)

            image = cv2.GaussianBlur(image,(5,5),0)
            image = cv2.GaussianBlur(image,(5,5),0)
            image = cv2.GaussianBlur(image,(5,5),0)
            image = cv2.GaussianBlur(image,(5,5),0)
            image = cv2.GaussianBlur(image,(5,5),0)
            image = cv2.GaussianBlur(image,(5,5),0)
            kernel = np.ones((3,3),np.uint8)
            resized = cv2.resize(image,self.target_size, interpolation = cv2.INTER_CUBIC)

        return resized

    def generate_scene_image(self, scene):  
        channels = []
        for i in range(scene.size(1)):
            agent_image = np.zeros((self.sampling_resolution[1], self.sampling_resolution[0]), np.uint8)
            agent = scene[:, i, :]
            channel = self.generate_trajectory_image(agent, agent_image)
            channel = np.array(channel)
            channels.append(channel)
        scene_image = np.stack(channels, axis=2)
        return scene_image
    
    def generate_batch_images(self, traj_batch):
        batch_size = traj_batch.size(1) // self.agent_num
        batch_images = []
        for i in range(batch_size):
            scene = traj_batch[:, i*self.agent_num: (i+1) * self.agent_num, :]
            scene_image = self.generate_scene_image(scene)
            batch_images.append(scene_image)
        batch_images = np.stack(batch_images)
        batch_images = torch.Tensor(batch_images)
        batch_images = batch_images.permute(0, 3, 1, 2)
        return batch_images
# drawer = TrajectoryDrawer()
# image = drawer.generate_batch_images(obs_traj)
# image.size()
# image.shape
# plt.imshow(image[:,:,:1], cmap="gray")
# plt.show()

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class LNet(nn.Module):
    def __init__(self, n_channels, bilinear=True):
        super(LNet, self).__init__()
        self.n_channels = n_channels
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        return x5
 
    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [104]:
class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, mask=None):

        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)

        return output, attn

class AttentionModule(nn.Module):
    """
    Query: Image
    Key-Value: Hidden States
    """
    def __init__(self, image_dim, hidden_dim, n_head=16, key_dim=16, value_dim=16, dropout=0.5):
        super(AttentionModule, self).__init__()
        
        self.image_dim = image_dim
        self.hidden_dim = hidden_dim
        self.key_dim = key_dim
        self.value_dim = value_dim
        self.n_head = n_head
        self.w_qs = nn.Linear(hidden_dim, n_head * key_dim, bias=False)
        self.w_ks = nn.Linear(image_dim, n_head * key_dim, bias=False)
        self.w_vs = nn.Linear(image_dim, n_head * value_dim, bias=False)
        self.fc = nn.Linear(n_head * value_dim, hidden_dim, bias=False)
        self.dot_attention = ScaledDotProductAttention(temperature=key_dim ** 0.5)
        
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim, eps=1e-6)

    
    def forward(self, images, hiddens):
        n_head = self.n_head
        key_dim = self.key_dim
        value_dim = self.value_dim
        batch_size = images.size(0)
        image_dim =images.size(1)
        k = images.view(batch_size, image_dim, -1)
        k = k.permute(0, 2, 1)
        len_image = k.size(1)
        
        q = hiddens
        residual = q
        len_hidden = 11
        v = k
        residual = hiddens
        q = self.w_qs(q).view(batch_size, len_hidden, n_head, key_dim)
        k = self.w_ks(k).view(batch_size, len_image, n_head, key_dim)
        v = self.w_vs(v).view(batch_size, len_image, n_head, value_dim)
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
        q, attn = self.dot_attention(q, k, v)
        q = q.transpose(1, 2).contiguous().view(batch_size, len_hidden, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)
        return q


class PostAttentionFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in=32, d_hid=128, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid) # position-wise
        self.w_2 = nn.Linear(d_hid, d_in) # position-wise
        self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)

        return x

class ImageAttentionLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, image_dim, hidden_dim, d_inner=128, n_head=16, key_dim=16, value_dim=16, dropout=0.5):
        super(ImageAttentionLayer, self).__init__()
        self.attn = AttentionModule(image_dim=image_dim, hidden_dim=hidden_dim, n_head=n_head, key_dim=key_dim, value_dim=value_dim, dropout=dropout)
        self.post_ffn = PostAttentionFeedForward(hidden_dim, d_inner, dropout=dropout)

    def forward(self, images, hiddens):
        attn_output = self.attn(
            images, hiddens)
        layer_output = self.post_ffn(attn_output)
        return  images, layer_output


In [126]:
import torchvision.models as models
from sgan.models import Encoder, Decoder, get_noise
class TrajectoryModel(nn.Module):
    def __init__(self, pred_len=8, encoder_h_dim=32, decoder_h_dim = 32, noise_dim=(0, ), noise_type='gaussian', noise_mix_type='ped'):
        super(TrajectoryModel, self).__init__()
        self.pred_len = pred_len
        self.image_drawer = TrajectoryDrawer(target_size=[224, 224])
        
        self.image_feature_extractor = self._make_image_feature_extractor()
        self.encoder = Encoder(h_dim=encoder_h_dim)
        self.num_layers = 1
        self.decoder_h_dim = decoder_h_dim
        self.encoder_h_dim = encoder_h_dim
        self.noise_mix_type = noise_mix_type
        self.noise_dim = noise_dim
        self.noise_type = noise_type
        
        self.attentions = nn.ModuleList([ImageAttentionLayer(hidden_dim=encoder_h_dim, image_dim=512) for i in range(2)])
        self.to_spatial = nn.Linear(encoder_h_dim, 2 * pred_len)
#         self.decoder = Decoder(
#             pred_len,
#             embedding_dim=embedding_dim,
#             h_dim=decoder_h_dim,
#             mlp_dim=mlp_dim,
#             num_layers=num_layers,
#             pool_every_timestep=pool_every_timestep,
#             dropout=dropout,
#             bottleneck_dim=bottleneck_dim,
#             activation=activation,
#             batch_norm=batch_norm,
#             pooling_type=pooling_type,
#             grid_size=grid_size,
#             neighborhood_size=neighborhood_size,)
    def _make_image_feature_extractor(self):
        resnet = models.resnet18()
        conv1 = nn.Conv2d(11, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        module_list = list(resnet.children())
        image_feature_extractor = nn.Sequential(conv1, *module_list[1:-2])
        
        return image_feature_extractor
    def add_noise(self, _input, seq_start_end, user_noise=None):
        """
        Inputs:
        - _input: Tensor of shape (_, decoder_h_dim - noise_first_dim)
        - seq_start_end: A list of tuples which delimit sequences within batch.
        - user_noise: Generally used for inference when you want to see
        relation between different types of noise and outputs.
        Outputs:
        - decoder_h: Tensor of shape (_, decoder_h_dim)
        """
        if not self.noise_dim:
            return _input

        if self.noise_mix_type == 'global':
            noise_shape = (seq_start_end.size(0), ) + self.noise_dim
        else:
            noise_shape = (_input.size(0), ) + self.noise_dim

        if user_noise is not None:
            z_decoder = user_noise
        else:
            z_decoder = get_noise(noise_shape, self.noise_type)

        if self.noise_mix_type == 'global':
            _list = []
            for idx, (start, end) in enumerate(seq_start_end):
                start = start.item()
                end = end.item()
                _vec = z_decoder[idx].view(1, -1)
                _to_cat = _vec.repeat(end - start, 1)
                _list.append(torch.cat([_input[start:end], _to_cat], dim=1))
            decoder_h = torch.cat(_list, dim=0)
            return decoder_h

        decoder_h = torch.cat([_input, z_decoder], dim=1)

        return decoder_h
    
    
    def forward(self, obs_traj, obs_traj_rel, seq_start_end, obs_team, obs_pos, user_noise=None,):
        batch_size = obs_traj_rel.size(1) // 11
        images = self.image_drawer.generate_batch_images(obs_traj.cpu())
        images = images.cuda()
        images.requires_grad=False
        
        image_features = self.image_feature_extractor(images)
        image_features = torch.squeeze(image_features)
        final_encoder_h = self.encoder(obs_traj_rel)
        hiddens = torch.squeeze(final_encoder_h)
        hiddens = hiddens.view(batch_size, 11, -1)
        
        
        for attention_layer in self.attentions:
            _, h = attention_layer(image_features, hiddens)
        spatial = self.to_spatial(h).view(batch_size * 11, -1)
        spatial = spatial.view(-1, self.pred_len, 2)
        spatial = spatial.permute(1, 0, 2)
#         mlp_decoder_context_input = final_encoder_h.view(
#                 -1, self.encoder_h_dim)
#         noise_input = mlp_decoder_context_input
#         decoder_h = self.add_noise(
#             noise_input, seq_start_end, user_noise=user_noise)
        
#         decoder_h = torch.unsqueeze(decoder_h, 0)
#         decoder_c = torch.zeros(
#             self.num_layers, batch_size, self.decoder_h_dim
#         ).cuda()

#         state_tuple = (decoder_h, decoder_c)
#         last_pos = obs_traj[-1]
#         last_pos_rel = obs_traj_rel[-1]
#         # Predict Trajectory

#         decoder_out = self.decoder(
#             last_pos,
#             last_pos_rel,
#             state_tuple,
#             seq_start_end,
#         )
        
        return spatial

In [84]:
batch = [tensor.cuda() for tensor in batch]
(obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, obs_team_vec, obs_pos_vec, pred_team_vec, pred_pos_vec, non_linear_ped, loss_mask, seq_start_end) = batch
drawer = TrajectoryDrawer(target_size=[224, 224])
images = drawer.generate_batch_images(obs_traj)
images = images.cuda()


In [127]:
# torch.empty_cache()
model = TrajectoryModel()
model = model.cuda()
r = model(obs_traj, obs_traj_rel, seq_start_end, obs_team_vec, obs_pos_vec)
r.size()

torch.Size([8, 176, 2])

In [39]:
176/ 16

11.0